3b)

In [67]:
# importing the required libraries
import numpy as np
import pandas as pd
# I have used my google drive to load the training and testing data
# Upload the dataset file in your google drive and change the path to run the below line
# Using the dataset of 2a of howework 1 only in 3b problem
from google.colab import drive
drive.mount('/content/drive')
train_data= pd.read_csv('/content/drive/MyDrive/ML/Priyadataset.csv',dtype='object')
print(train_data)
test_data=pd.read_csv('/content/drive/MyDrive/ML/PriyaTestdata.csv',dtype='object')
print(test_data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         height     diameter       weight          hue      class
0   0.117820005  0.138365287  0.376838312  3.255943211   Plastic 
1   0.149398845  0.137884636  0.458635784  4.887697997   Plastic 
2   0.092846989   0.08946483  0.398115904  3.314872309     Metal 
3   0.063493171         0.03  0.110605349  4.290583456     Metal 
4   0.132147759         0.15  0.366192424  2.642717324   Plastic 
5   0.134386556         0.15         0.75  2.549392832   Ceramic 
6   0.077117679  0.030795497  0.132188862  1.061354957     Metal 
7   0.088096721  0.093578829  0.514391815  2.807311094   Ceramic 
8   0.088439453   0.13763057  0.709234615  2.234895655   Ceramic 
9   0.146148564  0.135840972  0.451317027  4.177574005   Plastic 
10   0.09265452  0.080861592  0.567104482  1.641808644   Ceramic 
11  0.080698615  0.034717811          0.1   1.79095088   Plastic 
             

In [68]:
X = train_data.iloc[:,:-1]
Y = train_data.iloc[:,-1]
x_train = X.values
y_train = Y.values.reshape(-1,1)
x_train=x_train.astype('float64')
X_t= test_data.iloc[:,:-1]
x_test = X_t.values
x_test= x_test.astype('float64')

Implementing the decision tree

In [69]:
# defining the node ( basically each tree node)
# A = attribute  
# T = threshold
# l = left node
# r = right node
# Ig = information gain
# V = value
class Node():
    def __init__(self, A=None, T=None, l=None, r=None, Ig=None, V=None):
        self.A = A
        self.T = T
        self.l = l
        self.r= r
        self.Ig = Ig
        self.V = V

In [70]:
# decision tree implementation
class DecisionTree():
  # initializing the minimum sample default value as 2 and maximum depth value as 8
    def __init__(self, minimum_sample=2, maximum_depth=8):
        self.root = None
        self.minimum_sample = minimum_sample
        self.maximum_depth = maximum_depth
   
    # caluculating the entropy for the given value with log base of 10
    def Calculate_entropy(self, y_value):
        y_label = np.unique(y_value)
        #print(y_label)
        entropy = 0
        # for each attribute in the y_label
        for att in y_label:
            att_p = len(y_value[y_value == att]) / len(y_value)
            entropy += -att_p * np.log10(att_p)
            #print(entropy)
        # return the entropy value
        return entropy   

    # calculating the information gain in this function
    def calulate_info_gain(self, data, l_, r_):
       #caluclate the total datapoints in dataset 
        tot = len(l_) + len(r_)
        # how many datapoints in left tree / total length
        prob_left= len(l_) / tot
         # how many datapoints in right tree / total length
        prob_right = len(r_) / tot
        # calculate the left and right entropy by calling the function
        entropy_left = self.Calculate_entropy(l_)
        entropy_right= self.Calculate_entropy(r_)
        # value to split the data
        #print(entropy_left)
        #print(entropy_right)
        split_data = (prob_left * entropy_left) + (prob_right * entropy_right)
        #print(self.Calculate_entropy(data))
        # calculating the info gain
        infogain = self.Calculate_entropy(data) - split_data
        #print(infogain)
        return infogain

    # best split is calculated by calculating the thershold value by taking the best average of 2 data points in whole dataset
    #  returns the best split thershold value node
    def best_split(self, data, rows, cols):
        best_s = {}
        #initialzing the information gain to infinity 
        info_gain = -float('inf')
        for col in range(cols):
            value = data[:, col]
            _t = []
            #print(_t)
            # calulating the thershold by taking 2 datapoints and there average aand storing in _t variable
            for i in range(len(value)):
                if i != 0:
                    current = value[i]
                    next = value[i - 1]
                    split = (current + next) / 2
                    _t.append(split)
            #print(_t)
            # spliting the data based on each thershold value and calculating which split gives the maximum information gain
            for t in _t:
              # divide the dataset based on the bestsplit  e 
                left = np.array([i for i in data if i[col]<=t])
                #print(left)
                right = np.array([i for i in data if i[col]>t])
                #print(right)
                if len(left)>0 and len(right)>0:
                    y, left_y, right_y = data[:, 0:4], left[:, 0:4], right[:, 0:4]
                    # calling information gain method to check the value
                    IG = self.calulate_info_gain(y, left_y, right_y)
                    #print(IG)
                    if IG>info_gain:
                        best_s["A"] = col
                        best_s["T"] = t
                        best_s["l"] = left
                        best_s["r"] = right
                        best_s["Ig"] = IG
                        info_gain = IG 
            _t = []
            #print(_t)
            #print(best_s)
        return best_s

    # building the tree 
    # dividing the into left and right tree  by criteria of maximum information gain and returing the node
    def tree_build(self, data, depth=0):        
        x_data, y_data = data[:,0:4], data[:,4]
        rows, cols = np.shape(x_data)
        # checking whether the minimum sample and maximum depth number is satisfied
        if rows>=self.minimum_sample and depth<=self.maximum_depth:
          # calling the best split function to divide it into left and right tree
            best_s = self.best_split(data, rows, cols)
            if best_s["Ig"]>0:
               # using the bestsplit data left and right tree is build by calling the function recursively
                left_tree = self.tree_build(best_s["l"], depth+1)
                right_tree = self.tree_build(best_s["r"], depth+1)
                return Node(best_s["A"], best_s["T"], left_tree, right_tree, best_s["Ig"])
        # after continous manipulation of left and right tree --> we come with the left node
        leaf_n = max(list(y_data), key=list(y_data).count)
        #print(left_n)
        return Node(V=leaf_n)

    # training the dataset for decision tree
    def fit_funct(self, X_value, Y_value):
        whole_dataset = np.concatenate((X_value, Y_value), axis=1)
        # calling the function tree_build to build the decision tree
        self.root = self.tree_build(whole_dataset)
    
    # prediction function calls the classification to traverse through the built tree
    def class_predict(self, x_value):
      # initially it is started with root node hence self.root is passed
        pred = [self.classification(x, self.root) for x in x_value]
        return pred

    # classification function helps to traverse through the tree to match the attribute  and predict the required output
    def classification(self, x_v, d_tree):
      # for 1 level tree
        if d_tree.V!=None: return d_tree.V
        #checking for the attribute values
        value = x_v[d_tree.A]
        #print(value)
        # using <= value to divide the threshold value to left or right tree
        if value <= d_tree.T:
            return self.classification(x_v, d_tree.l)
        else:
            return self.classification(x_v, d_tree.r)
    
    # Calculating the accuracy value using the predicted value and actual value
    def accuracy(self, y_test, y_pred):
      correct = 0
      for i in range(len(y_test)):
        if y_test[i] == y_pred[i]:
            correct += 1
      #print(correct)
      return correct/float(len(y_test)) * 100.0 

Building the decision tree based on the predetermined depth of 3



In [71]:
print("the pre-determined depth considered is 3")
dec_tree = DecisionTree(minimum_sample=2, maximum_depth=3)
dec_tree.fit_funct(x_train,y_train)
y_pred = dec_tree.class_predict(x_train)
print('Accuracy for the training data: ',dec_tree.accuracy(y_train, y_pred))
y_pred_test = dec_tree.class_predict(x_test)
print('Values predicted for the test data:', y_pred_test)

the pre-determined depth considered is 3
Accuracy for the training data:  100.0
Values predicted for the test data: [' Metal ', ' Ceramic ', ' Metal ', ' Plastic ']


3c)

deriving trees of depths 1 until 8 and evaluating the accuracy of
each of the 8 resulting trees for the training samples and for the test set

In [72]:
# dividing the large training data set into a test set
# containing the first 6 data items of each material type and a training set containing the remaining data points
# stored in the form of csv in the google drive
# I have used my google drive to load the training and testing data
# Upload the dataset file in your google drive and change the path to run the below line
train_data= pd.read_csv('/content/drive/MyDrive/ML/Priyadataset3cTrain.csv',dtype='object')
print(train_data)
test_data=pd.read_csv('/content/drive/MyDrive/ML/Priyadataset3cTest.csv',dtype='object')
print(test_data)

          height     diameter       weight          hue      class
0    0.081220302  0.030216702  0.143960586  3.269462663   Plastic 
1    0.123647141  0.116415097         0.75  1.972557371   Ceramic 
2    0.089885502  0.074007617  0.243078037  2.245829339   Plastic 
3    0.102275618  0.105571657  0.675379468  3.014154131   Ceramic 
4    0.130108528  0.126018429         0.75  2.568700685   Ceramic 
..           ...          ...          ...          ...        ...
98   0.153248087  0.130163833  0.401184574  2.179239163   Plastic 
99   0.120215228  0.116073582  0.338827775  0.876866304   Plastic 
100  0.075569171  0.072067954    0.1616056   3.33827839   Plastic 
101  0.131371291  0.091788908  0.278141251  2.383392581   Plastic 
102   0.12825405  0.148201394  0.455335366  2.686607561   Plastic 

[103 rows x 5 columns]
         height     diameter       weight          hue      class
0   0.124937779  0.131903758  0.386168249  2.951766979   Plastic 
1   0.144921374  0.125007466         0.7

In [73]:
X = train_data.iloc[:,:-1]
Y = train_data.iloc[:,-1]
x_train = X.values
y_train = Y.values.reshape(-1,1)
x_train=x_train.astype('float64')
X_t= test_data.iloc[:,:-1]
Y_t = test_data.iloc[:,-1]
x_test = X_t.values
y_test = Y_t.values.reshape(-1,1)
x_test= x_test.astype('float64')

In [74]:
for i in range(1, 9):
    print('For Depth value : ', i)
    dt = DecisionTree(minimum_sample=2, maximum_depth=i)
    dt.fit_funct(x_train,y_train)
    y_pred = dt.class_predict(x_train)
    print('Accuracy on training dataset: ', dt.accuracy(y_train, y_pred))   
    y_pred_test = dt.class_predict(x_test)
    print('Accuracy on test dataset: ', dt.accuracy(y_test, y_pred_test))

For Depth value :  1
Accuracy on training dataset:  69.90291262135922
Accuracy on test dataset:  50.0
For Depth value :  2
Accuracy on training dataset:  69.90291262135922
Accuracy on test dataset:  50.0
For Depth value :  3
Accuracy on training dataset:  73.7864077669903
Accuracy on test dataset:  61.111111111111114
For Depth value :  4
Accuracy on training dataset:  79.6116504854369
Accuracy on test dataset:  55.55555555555556
For Depth value :  5
Accuracy on training dataset:  84.46601941747572
Accuracy on test dataset:  55.55555555555556
For Depth value :  6
Accuracy on training dataset:  100.0
Accuracy on test dataset:  55.55555555555556
For Depth value :  7
Accuracy on training dataset:  100.0
Accuracy on test dataset:  55.55555555555556
For Depth value :  8
Accuracy on training dataset:  100.0
Accuracy on test dataset:  55.55555555555556


By comparing the accuracy of training and test dataset we can say that the best decision tree is with the depth 3 because it has the maximum test data accuracy of 61.111% 

As we can see depth 3 accuracy of test data is 61.11%, increasing the depth value led to the decrease in the test data accuracy 

so from this we can conclude that from depth 4 onwards the decision tree tried to overfit 